In [1]:
import numpy as np
from tfrecords2numpy_qc import TFRecordsParser
from tfrecords2numpy_qc import TFRecordsElevation
import os
import pickle
from tqdm import tqdm
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler
import webdataset as wds
import xarray as xr
import h5py
import random
from collections import Counter

In [5]:
PATH_TO_FORCING = "/glade/scratch/yiwenz/CESM_0.125_US_cities/forcing_Phoenix_Mesa_AZ.pkl"
PATH_TO_DATA = "/glade/scratch/yiwenz/Data_TFRecord_Daily/"
PATH_TO_STORE = "/glade/scratch/yiwenz/transfer_learning_hdf5/phoenix_data.h5"
PATH_TO_LATLONG = "/glade/scratch/yiwenz/modis_lat_lon_US_cities/modis_grid_Phoenix.csv"
PATH_TO_ELEVATIONS = "/glade/work/yiwenz/AWS3D30_cropped.tfrecord"

channel_choices = ['Red', 'Green', 'Blue', "NIR", "SWIR1"]
threshold_cloud=0.15

In [6]:
def inverse_distance_mean():
    latlong = pd.read_csv(PATH_TO_LATLONG, usecols=["lon", "lat"])
    array = latlong.values
    dist = euclidean_distances(array, array)
    weights_idx_dict = {}
    for idx in range(len(dist)):
        sort_idx = dist[idx, :].argsort()[1:8]
        closest = dist[idx, sort_idx]**-1
        closest = closest / closest.sum()
        weights_idx_dict[idx] = {"weights":closest, "idx":sort_idx}

    return weights_idx_dict

def extract_data(path):
    """
    Extract all data from all TFRecords files and stores as pickled tuples in the format (image, label)
    :return:
    Each Sample is of Array Size 8721. For a single sample arr, this is how we can access our data:
    arr[:7] -> 7 Forcing Variables
    arr[7] -> Month Indicator of Data
    arr[8] -> LST Target
    arr[9:].reshape(8,33,33) -> Sattelite Image Tensor 
    """
    tot_samples = 0
    first_batch = True
    with h5py.File(path, "a") as hp5:
        center_weight = 1
        outside_weight = 1 - center_weight
        weights_idx_dict = inverse_distance_mean()
        elevations_dict = TFRecordsElevation(filepath=PATH_TO_ELEVATIONS).tfrecrods2numpy()
        with open(PATH_TO_FORCING, 'rb') as f:
            forcing_data = pickle.load(f)

        avail_dates = list(forcing_data.keys())

        for root, dirs, files in os.walk(PATH_TO_DATA):
            random.shuffle(files)
            for file in tqdm(files, desc="Total Progress"):
                samples = []
                file_date = file.split(".")[0] 
                if file_date in avail_dates:
                    print(file_date)
                    path_to_tf = os.path.join(PATH_TO_DATA, file)
                    records = TFRecordsParser(path_to_tf, channels=channel_choices).tfrecrods2numpy()
                    all_pixels = len(records)
                    cloud_pixels = Counter(elem[1] for elem in records)[False]
                    cloud_pct = cloud_pixels/all_pixels
                    month = int(file_date[2:4])
                    if cloud_pct<=threshold_cloud:
                        print(f'Meet cloud threshold. Cloud cover pct: {cloud_pct}')
                        for idx, (features, lst, qc_flag) in enumerate(records):
                            if (qc_flag is not False) and (qc_flag>>1&1)==0 and (qc_flag>>7&1)==0 and (lst is not False) and ((features!=-9999).all() == True):
                                if outside_weight != 0:
                                    weights = np.array(weights_idx_dict[idx]["weights"]) * outside_weight
                                    min_idx = weights_idx_dict[idx]["idx"]
                                    surround_features = np.array([records[i][0] for i in min_idx])
                                    weighted_avg = np.average(surround_features, weights=weights, axis=0)
                                    features = np.average([features, weighted_avg], weights=[center_weight, outside_weight], axis=0)
                                NIR_dn = (features[3]+0.2)/2.75e-05
                                SWIR1_dn = (features[4]+0.2)/2.75e-05
                                RED_dn = (features[0]+0.2)/2.75e-05

                                features_ndbi = ((SWIR1_dn-NIR_dn)/(SWIR1_dn+NIR_dn)).reshape(-1,33,33)
                                features_ndvi = ((NIR_dn-RED_dn)/(NIR_dn+RED_dn)).reshape(-1,33,33)
                                features = np.concatenate([features,features_ndbi,features_ndvi],axis=0)
                                elevations = elevations_dict[idx].reshape(-1,33,33)
                                features = np.vstack([features, elevations])
                                forcing = forcing_data[file_date][idx]

                                sample_image = features.flatten()
                                month = np.array(month).reshape(-1,)
                                lst = np.array(lst).reshape(-1,)

                                ex_array = np.concatenate((forcing, month, lst, sample_image))

    #                             VALIDATION CHECKS
    #                             reb_forcing = ex_array[:7]
    #                             reb_month = ex_array[7]
    #                             reb_lst = ex_array[8]
    #                             reb_image = ex_array[9:].reshape(8,33,33)

    #                             print(reb_forcing == forcing)
    #                             print(reb_month == month)
    #                             print(reb_lst == lst)
    #                             print((reb_image == features).all())

                                samples.append(ex_array)
                samples = np.array(samples)
                new_samples_num = samples.shape[0]
                if new_samples_num > 0:
                    ## ONLY APPEND IF DATA EXISTS ##
                    tot_samples += new_samples_num
                    np.random.shuffle(samples)
                    _, feature_length = samples.shape
                    assert(feature_length == 8721)
                    if first_batch:
                        hdf5_dataset = hp5.create_dataset('phoenix', (new_samples_num, feature_length), maxshape=(None, feature_length), dtype='float32')
                        hdf5_dataset[:] = samples
                        first_batch = False

                    hdf5_dataset.resize(tot_samples, axis=0)
                    hdf5_dataset[-new_samples_num:] = samples
    return "DONE"
                

In [ ]:
extract_data(PATH_TO_STORE)

Total Progress:   0%|          | 0/6024 [00:00<?, ?it/s]

180110


Total Progress:   0%|          | 1/6024 [00:06<10:37:17,  6.35s/it]

140928


Total Progress:   0%|          | 2/6024 [00:12<10:33:49,  6.32s/it]

...

120413


Total Progress:   1%|          | 75/6024 [07:46<10:14:02,  6.19s/it]

040315


Total Progress:   1%|▏         | 76/6024 [07:53<10:13:58,  6.19s/it]

Meet cloud threshold. Cloud cover pct: 0.0
060110


In [ ]:
f = h5py.File(PATH_TO_STORE, 'r')